In [ ]:
!pip install "shapely<2.0.0"
! pip install google-cloud-aiplatform langchain chromadb pydantic typing-inspect typing_extensions pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# LangChain

In [ ]:
import vertexai
import langchain
from langchain.llms import VertexAI
from google.cloud import aiplatform

PROJECT_ID = "vertext-ai-dar"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")


In [ ]:
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.8,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

llm("Tell me a joke")

In [ ]:
prompt_1 = "List the key dates in the life of Steve Jobs"

prompt_2 = """
    Classify the following question as one of the following: Computers, Audio-Video, or Appliances.

    Question: Show me your TVs.
    Answer: Audio-Video

    Question: Do you have any good deals on PCs?
    Answer:
"""

prompt_3 = """
    Context: You answer questions about dogs.

    If someone asks a questions about cats just return "Woof, I don't know"

    Q: What is a good breed of dog for kids?
    A: Golden Retrieves are nice. There are lots of of good dogs too.

    Q: What is the best treat for cats?
    A: Woof, I don't know

    Q: How can I get my cat to stop attacking my dog?
    A:

"""

llm(prompt_1)

# Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Context: You write in the style of {style}.
    Write me a {output} about {thing}.
    """
)
prompt_template.format(style="a pirate", output="poem", thing="COBOL Programming")

In [ ]:
llm(prompt_template.format(style="a pirate", output="poem", thing="COBOL Programming"))

# Custom Prompt Template

In [ ]:
def rb(arg):
    r = 0
    b = arg.bit_length()

    for i in range(b):
        r <<= 1
        r |= (arg & 1)
        r >>= 1

    return r

In [ ]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator
import inspect

PROMPT = """
Given the function name and source code, generate an English language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input,
    and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = inspect.getsource(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

    def _prompt_type(self):
        return "function-explainer"

In [ ]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=rb)
print(prompt)

In [ ]:
llm(prompt)

# Prompt Template Pipelining

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

prompt = (
    PromptTemplate.from_template("""Tell me a joke about {topic},
    make it funny and in {language}""")
)

chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"topic": "COBOL", "language": "English"})


# Output Parser

In [ ]:
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "Tell me a joke about Python programming."})

parser.invoke(output)

In [ ]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=actor_query)

output = llm(_input.to_string())

parser.parse(output)

# List Parser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="""List five {subject}, Only list the items with no formatting.
    {format_instructions}""",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

_input = prompt.format(subject="ice cream flavors")
output = llm(_input)

output_parser.parse(output)

In [ ]:
# Here's another example, but with a compound typed field.
class Athlete(BaseModel):
    name: str = Field(description="name of an athlete")
    teams: List[str] = Field(description="list of teams they played for")


athlete_query = "Name a random NFL Quarterback from the 1900s."

parser = PydanticOutputParser(pydantic_object=Athlete)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=athlete_query)

output = llm(_input.to_string())

parser.parse(output)

# LCEL

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt = (
    PromptTemplate.from_template("Write me a poem about {topic}"
    + ", make it rhyme"
    + "\n\nand in {language}")
)



# LCEL Syntax
chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"topic": "COBOL", "language": "English"}):
    print(chunk, end="", flush=True)

# Chat

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatVertexAI(model_name="chat-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,)

chat([HumanMessage(content="What's a good recipe for a Halloween Party?")])

In [ ]:
response = chat(
    [
        SystemMessage(content="You are a bot who knows about cooking"),
        HumanMessage(content="What's a good desert for Thanksgiving"),
        AIMessage(content="Pumpkin pie is always a winner."),
        HumanMessage(content="Great, what is the recipe?")
    ]
)
print(response)

# ChatPromptTemplate

In [ ]:
from langchain.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful {job}. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
messages = chat_template.format_messages(
                      job="Chef",
                      name="Julia",
                      user_input="What is your name and what do you do?")

chat(messages)

# Chat with Memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=chat, memory=memory, verbose=False)

input = """
    System: You are a Chef named Julia.
    Human: What is a good recipe for dinner that includes bananas?
"""

conversation.predict(input = input)

In [ ]:
conversation.predict(input = "How long would that take to prepare?")

In [ ]:
conversation.predict(input="Should those be served warm or chilled")

In [ ]:
print(memory.buffer)

In [ ]:
from langchain.embeddings import VertexAIEmbeddings

# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [ ]:

# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)